In [101]:
from notebook.notebookapp import list_running_servers
import requests
import json

In [105]:
servers = list(list_running_servers())
for s in servers:
    print(s)
lk = 'http://localhost:8888'+servers[0]['base_url']+'api/sessions'
r = requests.get(lk)
# json.loads(r.text)

{'base_url': '/', 'hostname': 'localhost', 'notebook_dir': '/Users/eyam/jupyter/jupyterLabIDE/submit_jobs', 'password': False, 'pid': 57269, 'port': 8888, 'secure': False, 'token': '', 'url': 'http://localhost:8888/'}


In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%pyth

In [2]:
%load_ext submit_jobs.hysds_magic

AttributeError: module 'submit_jobs.hysds_magic' has no attribute '__file__'

In [5]:
%ls submit_jobs/hysds_magic

hysds.py


In [97]:
from IPython.core.magic import (Magics, magics_class, line_magic, cell_magic)
import requests
import json

@magics_class
class HysdsMagic(Magics):

    @line_magic
    def execute(self, line):
        lk = 'http://localhost:8888'
        endpoint = '/hysds/execute'
        id = 'org.n52.wps.server.algorithm.SimpleBufferAlgorithm'
        algo_ver,params = line.split('(')
        params = params[:-1]
        algo,ver = algo_ver.split(':')
        inputs = []
        for kvpair in params.split(','):
            k,v = kvpair.split('=')
            inputs.append(k)
        inputs.append('')
        call = '?algo_id={algo}&version={ver}&identifier={id}&inputs={inputs}&{params}'.format(algo=algo,ver=ver,id=id,inputs=','.join(inputs),params=params.replace(',','&'))
        url = lk + call
        r = requests.get(url)
        resp = json.loads(r.text)
        print('{}\n{}'.format(url,resp['result']))
        return 

    @cell_magic
    def cadabra(self, line, cell):
        return line, cell


In [98]:
ip = get_ipython()
m = HysdsMagic()
ip.register_magics(m)

In [99]:
# %execute algo:version(p1=v1,p2=v2)
%execute plant_test:master(localize_urls=[],username=liz)

http://localhost:8888/hysds/execute?algo_id=plant_test&version=master&identifier=org.n52.wps.server.algorithm.SimpleBufferAlgorithm&inputs=localize_urls,username,&localize_urls=[]&username=liz
JobID is ea6cc805-3cc5-41fb-a42d-7ad20aebec28
